<a href="https://colab.research.google.com/github/Akshata1712/Federated-Learning-/blob/main/FedML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs (TabNet + XGBoost). This may take ~1-2 minutes.
!pip install -q xgboost pytorch-tabnet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Update this to your creditcard.csv location in Drive
DATA_PATH = "/content/drive/MyDrive/creditcard.csv"

# Standard imports
import os, random, time
import numpy as np, pandas as pd
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt

# For TabNet / XGBoost
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.ensemble import RandomForestClassifier

# Device + seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)


Mounted at /content/drive
Device: cuda


In [ ]:
!pip install data_utils

ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=2.6,<=3.0; 0.0.2 Requires-Python >=2.6,<=3.0; 0.0.3 Requires-Python >=2.6,<=3.0; 0.0.4 Requires-Python >=2.6,<=3.0; 0.0.5 Requires-Python >=2.6,<=3.0; 0.0.6 Requires-Python >=2.6,<=3.0; 0.0.7 Requires-Python >=2.6,<=3.0
ERROR: Could not find a version that satisfies the requirement data_utils (from versions: none)
ERROR: No matching distribution found for data_utils


In [ ]:
def load_and_preprocess(path=DATA_PATH, seed=SEED, test_size=0.10, distill_size=0.1111):
    df = pd.read_csv(path)
    X = df.drop("Class", axis=1).values
    y = df["Class"].values.reshape(-1,1).astype(np.float32)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=test_size, random_state=seed, stratify=y)
    X_train, X_distill, y_train, y_distill = train_test_split(
        X_train_full, y_train_full, test_size=distill_size, random_state=seed, stratify=y_train_full)
    # to tensors (torch) for torch models; for non-torch keep numpy copies too
    X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_t = torch.tensor(y_train, dtype=torch.float32).to(device)
    X_distill_t = torch.tensor(X_distill, dtype=torch.float32).to(device)
    y_distill_t = torch.tensor(y_distill, dtype=torch.float32).to(device)
    X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_t = torch.tensor(y_test, dtype=torch.float32).to(device)
    # also provide numpy versions for non-torch models
    return X_train_t, y_train_t, X_distill_t, y_distill_t, X_test_t, y_test_t, X_train, y_train, X_distill, y_distill, X_test, y_test

X_train, y_train, X_distill, y_distill, X_test, y_test, \
X_train_np, y_train_np, X_distill_np, y_distill_np, X_test_np, y_test_np = load_and_preprocess()
print("Shapes (torch tensors):", X_train.shape, y_train.shape, X_distill.shape, X_test.shape)
print("Train fraud rate:", float((y_train==1).float().mean()))


Shapes (torch tensors): torch.Size([227848, 30]) torch.Size([227848, 1]) torch.Size([28478, 30]) torch.Size([28481, 30])
Train fraud rate: 0.001729222945868969


In [ ]:
# Torch models (logits output)
class StdMLP(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(dim,64), nn.ReLU(), nn.Linear(64,1))
    def forward(self,x): return self.net(x)

class DeepMLP(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(dim,64), nn.ReLU(),
                                 nn.Linear(64,64), nn.ReLU(),
                                 nn.Linear(64,1))
    def forward(self,x): return self.net(x)

class WideMLP(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(dim,128), nn.ReLU(), nn.Linear(128,1))
    def forward(self,x): return self.net(x)

def count_params(model):
    return sum(p.numel() for p in model.parameters())

# Wrappers for non-torch models to produce probabilities on numpy arrays
class XGBWrap:
    def __init__(self, scale_pos_weight=1.0):
        self.model = XGBClassifier(scale_pos_weight=float(scale_pos_weight), use_label_encoder=False, eval_metric='logloss', n_jobs=1)
    def fit(self, X_np, y_np):
        self.model.fit(X_np, y_np.ravel())
    def predict_proba(self, X_np):
        # returns column vector of probabilities shape (N,1)
        return self.model.predict_proba(X_np)[:, 1].reshape(-1,1)

class TabNetWrap:
    def __init__(self):
        # small TabNet config (train short for speed)
        self.model = TabNetClassifier(n_d=16, n_a=16, n_steps=3, verbose=0)
    def fit(self, X_np, y_np):
        # TabNet expects 2D numpy arrays
        self.model.fit(X_np, y_np.ravel(), max_epochs=10, batch_size=256, virtual_batch_size=64, patience=5)
    def predict_proba(self, X_np):
        return self.model.predict_proba(X_np)[:,1].reshape(-1,1)

class RFWrap:
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=SEED)
    def fit(self, X_np, y_np):
        self.model.fit(X_np, y_np.ravel())
    def predict_proba(self, X_np):
        return self.model.predict_proba(X_np)[:,1].reshape(-1,1)


In [ ]:
# Local training for torch models with local validation split; returns model & local val F1
def local_train_and_eval_torch(model, X_t, y_t, epochs=1, batch_size=64, lr=0.01, pos_weight=None, val_frac=0.1):
    n = X_t.shape[0]
    if n < 4:
        X_tr, X_val = X_t, X_t
        y_tr, y_val = y_t, y_t
    else:
        val_n = max(1, int(val_frac * n))
        X_tr, X_val = X_t[:-val_n], X_t[-val_n:]
        y_tr, y_val = y_t[:-val_n], y_t[-val_n:]

    ds = TensorDataset(X_tr, y_tr)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True)
    model.train()
    if pos_weight is not None:
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight], device=device))
    else:
        criterion = nn.BCEWithLogitsLoss()
    opt = optim.SGD(model.parameters(), lr=lr)
    for _ in range(epochs):
        for xb, yb in loader:
            opt.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            opt.step()
    # val f1
    model.eval()
    with torch.no_grad():
        probs = torch.sigmoid(model(X_val))
        preds = (probs > 0.5).float()
    val_f1 = f1_score(y_val.cpu().numpy(), preds.cpu().numpy(), zero_division=0)
    return model, val_f1

# Training non-torch model wrapper with local validation: returns fitted model and val_f1
def local_train_and_eval_non_torch(wrapper, X_np, y_np, val_frac=0.1):
    n = X_np.shape[0]
    if n < 4:
        X_tr, X_val = X_np, X_np
        y_tr, y_val = y_np, y_np
    else:
        val_n = max(1, int(val_frac * n))
        X_tr, X_val = X_np[:-val_n], X_np[-val_n:]
        y_tr, y_val = y_np[:-val_n], y_np[-val_n:]
    wrapper.fit(X_tr, y_tr)
    # predict on validation
    probs = wrapper.predict_proba(X_val)
    preds = (probs > 0.5).astype(float)
    val_f1 = f1_score(y_val.ravel(), preds.ravel(), zero_division=0)
    return wrapper, val_f1

# Evaluate a logits model (torch) on test
def evaluate_logits(model, X_t, y_t):
    model.eval()
    with torch.no_grad():
        probs = torch.sigmoid(model(X_t))
        preds = (probs > 0.5).float()
    acc = accuracy_score(y_t.cpu().numpy(), preds.cpu().numpy())
    f1 = f1_score(y_t.cpu().numpy(), preds.cpu().numpy(), zero_division=0)
    bce = nn.BCELoss()
    with torch.no_grad():
        loss = bce(probs, y_t).item()
    return acc, f1, loss


In [ ]:
def create_splits(n_samples, num_clients):
    return [list(range(i, n_samples, num_clients)) for i in range(num_clients)]

def capacity_scores(models):
    # models: list of objects (torch models or wrappers). For wrappers that are not torch, we assign approximate capacity via hasattr('model') and .n_estimators etc.
    caps = []
    for m in models:
        if isinstance(m, nn.Module):
            caps.append(count_params(m))
        else:
            # heuristic capacities for classical models
            if isinstance(m, XGBWrap):
                # approximate capacity: number of trees * depth? can't access easily; use constant
                caps.append(10000)  # heuristic
            elif isinstance(m, TabNetWrap):
                caps.append(20000)
            elif isinstance(m, RFWrap):
                caps.append(15000)
            else:
                caps.append(10000)
    caps = np.array(caps, dtype=float)
    # log-normalize to 0..1
    logp = np.log(caps + 1.0)
    norm = (logp - logp.min()) / (logp.max() - logp.min() + 1e-12)
    return norm


In [ ]:
def run_homogeneous_fedavg(X_train_t, y_train_t, X_test_t, y_test_t,
                           num_clients=10, rounds=20, local_epochs=1, lr=0.01, batch_size=64):
    n = X_train_t.shape[0]
    splits = create_splits(n, num_clients)
    client_sizes = [len(s) for s in splits]
    pos_weight = (y_train_t==0).sum() / (y_train_t==1).sum()
    dim = X_train_t.shape[1]
    global_model = StdMLP(dim).to(device)
    accs, f1s, losses = [], [], []
    for r in range(rounds):
        local_states = []
        for i in range(num_clients):
            idx = splits[i]
            Xc, yc = X_train_t[idx], y_train_t[idx]
            client = StdMLP(dim).to(device)
            client.load_state_dict(global_model.state_dict())
            client, _ = local_train_and_eval_torch(client, Xc, yc, epochs=local_epochs, batch_size=batch_size, lr=lr, pos_weight=pos_weight)
            local_states.append({k: v.cpu() for k,v in client.state_dict().items()})
        # FedAvg
        total = sum(client_sizes)
        new_state = {}
        for k in local_states[0].keys():
            new_state[k] = sum(local_states[i][k] * client_sizes[i] for i in range(num_clients)) / total
        global_model.load_state_dict(new_state)
        acc, f1, loss = evaluate_logits(global_model, X_test_t, y_test_t)
        accs.append(acc); f1s.append(f1); losses.append(loss)
        print(f"[FedAvg] Round {r+1}/{rounds}: Acc={acc:.4f}, F1={f1:.4f}, BCE(test)={loss:.4f}")
    return accs, f1s, losses, global_model

# Quick run (adjust rounds/local_epochs as desired)
hom_accs, hom_f1s, hom_losses, hom_global = run_homogeneous_fedavg(X_train, y_train, X_test, y_test,
                                                                  num_clients=10, rounds=10, local_epochs=1)


[FedAvg] Round 1/10: Acc=0.9762, F1=0.1147, BCE(test)=0.3111
[FedAvg] Round 2/10: Acc=0.9803, F1=0.1356, BCE(test)=0.1761
[FedAvg] Round 3/10: Acc=0.9825, F1=0.1504, BCE(test)=0.1382
[FedAvg] Round 4/10: Acc=0.9841, F1=0.1661, BCE(test)=0.1196
[FedAvg] Round 5/10: Acc=0.9754, F1=0.1139, BCE(test)=0.1394
[FedAvg] Round 6/10: Acc=0.9787, F1=0.1289, BCE(test)=0.1245
[FedAvg] Round 7/10: Acc=0.9836, F1=0.1616, BCE(test)=0.1056
[FedAvg] Round 8/10: Acc=0.9781, F1=0.1261, BCE(test)=0.1177
[FedAvg] Round 9/10: Acc=0.9818, F1=0.1478, BCE(test)=0.1073
[FedAvg] Round 10/10: Acc=0.9833, F1=0.1593, BCE(test)=0.1014


In [ ]:
def run_hetero_mlp_distill(X_train_t, y_train_t, X_distill_t, y_distill_t, X_test_t, y_test_t,
                           client_archs, num_clients=10, rounds=20, local_epochs=1, lr=0.01, batch_size=64,
                           alpha=0.7, beta=0.3):
    """
    client_archs: list of length num_clients containing 'std'/'deep'/'wide'
    alpha,beta: weight for val_f1 and capacity in capacity-aware weighting
    """
    n = X_train_t.shape[0]
    splits = create_splits(n, num_clients)
    pos_weight = (y_train_t==0).sum() / (y_train_t==1).sum()
    dim = X_train_t.shape[1]

    # instantiate clients
    clients = []
    for arch in client_archs:
        if arch == 'std': clients.append(StdMLP(dim).to(device))
        elif arch == 'deep': clients.append(DeepMLP(dim).to(device))
        elif arch == 'wide': clients.append(WideMLP(dim).to(device))
        else: raise ValueError("unknown arch")

    # capacity norm
    cap_norm = capacity_scores(clients)  # 0..1 numpy

    accs_simple, f1s_simple, losses_simple = [], [], []
    accs_cap, f1s_cap, losses_cap = [], [], []

    for r in range(rounds):
        val_f1s = []
        # local train each client and get val_f1
        for i, m in enumerate(clients):
            idx = splits[i]
            Xc, yc = X_train_t[idx], y_train_t[idx]
            m, valf1 = local_train_and_eval_torch(m, Xc, yc, epochs=local_epochs, batch_size=batch_size, lr=lr, pos_weight=pos_weight)
            val_f1s.append(valf1)
        val_f1s = np.array(val_f1s, dtype=float)
        # --- Simple distillation: average probs equally ---
        with torch.no_grad():
            preds_sum = None
            for m in clients:
                out = torch.sigmoid(m(X_distill_t))   # shape [D,1]
                preds_sum = out.clone() if preds_sum is None else preds_sum + out
            avg_soft_simple = (preds_sum / len(clients)).clamp(0.0,1.0)

        # distill into global model (simple)
        global_simple = StdMLP(dim).to(device)
        optg = optim.SGD(global_simple.parameters(), lr=lr)
        bce = nn.BCELoss()
        for _ in range(local_epochs):
            optg.zero_grad()
            gp = torch.sigmoid(global_simple(X_distill_t))
            loss = bce(gp, avg_soft_simple)
            loss.backward(); optg.step()
        acc_s, f1_s, loss_s = evaluate_logits(global_simple, X_test_t, y_test_t)
        accs_simple.append(acc_s); f1s_simple.append(f1_s); losses_simple.append(loss_s)

        # --- Capacity-weighted distillation: weights from val_f1 and cap_norm ---
        scores = alpha * (val_f1s - val_f1s.min()) / (val_f1s.max()-val_f1s.min()+1e-12) + beta * cap_norm
        # normalize to sum=1
        if scores.sum() == 0:
            weights = np.ones_like(scores)/len(scores)
        else:
            weights = scores / scores.sum()

        with torch.no_grad():
            preds_weighted = None
            for m, w in zip(clients, weights):
                out = torch.sigmoid(m(X_distill_t))
                if preds_weighted is None:
                    preds_weighted = w * out
                else:
                    preds_weighted += w * out
            avg_soft_cap = preds_weighted.clamp(0.0,1.0)

        # distill into global model (capacity-weighted)
        global_cap = StdMLP(dim).to(device)
        optg2 = optim.SGD(global_cap.parameters(), lr=lr)
        for _ in range(local_epochs):
            optg2.zero_grad()
            gp2 = torch.sigmoid(global_cap(X_distill_t))
            loss2 = bce(gp2, avg_soft_cap)
            loss2.backward(); optg2.step()
        acc_c, f1_c, loss_c = evaluate_logits(global_cap, X_test_t, y_test_t)
        accs_cap.append(acc_c); f1s_cap.append(f1_c); losses_cap.append(loss_c)

        print(f"[Hetero-MLP] Round {r+1}/{rounds}: Simple Acc={acc_s:.4f}, F1={f1_s:.4f} | Cap-Weighted Acc={acc_c:.4f}, F1={f1_c:.4f}")

    return (accs_simple, f1s_simple, losses_simple, accs_cap, f1s_cap, losses_cap)

# Example client mix
client_archs = ['std']*4 + ['deep']*3 + ['wide']*3
hetmlp_results = run_hetero_mlp_distill(X_train, y_train, X_distill, y_distill, X_test, y_test,
                                        client_archs, num_clients=10, rounds=10, local_epochs=1)


[Hetero-MLP] Round 1/10: Simple Acc=0.3666, F1=0.0050 | Cap-Weighted Acc=0.9601, F1=0.0018
[Hetero-MLP] Round 2/10: Simple Acc=0.4723, F1=0.0058 | Cap-Weighted Acc=0.8167, F1=0.0173
[Hetero-MLP] Round 3/10: Simple Acc=0.4105, F1=0.0047 | Cap-Weighted Acc=0.8108, F1=0.0096
[Hetero-MLP] Round 4/10: Simple Acc=0.1107, F1=0.0036 | Cap-Weighted Acc=0.9683, F1=0.0066
[Hetero-MLP] Round 5/10: Simple Acc=0.0784, F1=0.0036 | Cap-Weighted Acc=0.1648, F1=0.0038
[Hetero-MLP] Round 6/10: Simple Acc=0.4416, F1=0.0045 | Cap-Weighted Acc=0.7673, F1=0.0018
[Hetero-MLP] Round 7/10: Simple Acc=0.0047, F1=0.0034 | Cap-Weighted Acc=0.3652, F1=0.0028
[Hetero-MLP] Round 8/10: Simple Acc=0.5579, F1=0.0030 | Cap-Weighted Acc=0.7003, F1=0.0084
[Hetero-MLP] Round 9/10: Simple Acc=0.4804, F1=0.0030 | Cap-Weighted Acc=0.9340, F1=0.0042
[Hetero-MLP] Round 10/10: Simple Acc=0.7918, F1=0.0003 | Cap-Weighted Acc=0.5772, F1=0.0022


In [ ]:
def run_hetero_finance_distill(X_train_t, y_train_t, X_distill_t, y_distill_t, X_test_t, y_test_t,
                               X_train_np, y_train_np, X_distill_np, y_distill_np, X_test_np, y_test_np,
                               client_wrappers, num_clients=3, rounds=20, local_epochs=5, lr=0.01,
                               alpha=0.7, beta=0.3):
    """
    client_wrappers: list of classes e.g. [XGBWrap, TabNetWrap, RFWrap]
    This creates num_clients=len(client_wrappers) clients.
    """
    assert len(client_wrappers) == num_clients
    n = X_train_t.shape[0]
    splits = create_splits(n, num_clients)
    pos_weight = (y_train_t==0).sum() / (y_train_t==1).sum()
    dim = X_train_t.shape[1]

    # instantiate wrappers
    clients = []
    for W in client_wrappers:
        if W == XGBWrap:
            clients.append(XGBWrap(scale_pos_weight=float(pos_weight.cpu().item())))
        elif W == TabNetWrap:
            clients.append(TabNetWrap())
        elif W == RFWrap:
            clients.append(RFWrap())
        else:
            clients.append(W())

    # compute capacities (heuristic) and normalize
    cap_norm = capacity_scores(clients)

    accs_simple, f1s_simple, losses_simple = [], [], []
    accs_cap, f1s_cap, losses_cap = [], [], []

    # Initialize global models once before rounds start
    global_simple = StdMLP(dim).to(device)
    global_cap = StdMLP(dim).to(device)
    bce = nn.BCELoss()

    for r in range(rounds):
        val_f1s = []
        # Local training of each wrapper on its split
        for i, wrapper in enumerate(clients):
            idx = splits[i]
            Xc_np, yc_np = X_train_np[idx], y_train_np[idx]
            wrapper, valf1 = local_train_and_eval_non_torch(wrapper, Xc_np, yc_np)
            val_f1s.append(valf1)
        val_f1s = np.array(val_f1s, dtype=float)

        # --- Simple averaging of soft labels ---
        preds_sum = None
        for wrapper in clients:
            p = wrapper.predict_proba(X_distill_np)  # numpy (D,1)
            if preds_sum is None:
                preds_sum = p.copy()
            else:
                preds_sum += p
        avg_soft_simple = torch.tensor((preds_sum / len(clients)).astype(np.float32)).to(device).clamp(0.0,1.0)

        # Distill into global simple model
        optg = optim.SGD(global_simple.parameters(), lr=lr)
        for _ in range(local_epochs):
            optg.zero_grad()
            gp = torch.sigmoid(global_simple(X_distill_t))
            loss = bce(gp, avg_soft_simple)
            loss.backward()
            optg.step()
        acc_s, f1_s, loss_s = evaluate_logits(global_simple, X_test_t, y_test_t)
        accs_simple.append(acc_s)
        f1s_simple.append(f1_s)
        losses_simple.append(loss_s)

        # --- Capacity-aware weighted averaging ---
        scores = alpha * (val_f1s - val_f1s.min()) / (val_f1s.max() - val_f1s.min() + 1e-12) + beta * cap_norm
        if scores.sum() == 0:
            weights = np.ones_like(scores) / len(scores)
        else:
            weights = scores / scores.sum()
        preds_w = None
        for wrapper, w in zip(clients, weights):
            p = wrapper.predict_proba(X_distill_np)
            if preds_w is None:
                preds_w = w * p.copy()
            else:
                preds_w += w * p.copy()
        avg_soft_cap = torch.tensor(preds_w.astype(np.float32)).to(device).clamp(0.0,1.0)

        # Distill into global capacity-weighted model
        optg2 = optim.SGD(global_cap.parameters(), lr=lr)
        for _ in range(local_epochs):
            optg2.zero_grad()
            gp2 = torch.sigmoid(global_cap(X_distill_t))
            loss2 = bce(gp2, avg_soft_cap)
            loss2.backward()
            optg2.step()
        acc_c, f1_c, loss_c = evaluate_logits(global_cap, X_test_t, y_test_t)
        accs_cap.append(acc_c)
        f1s_cap.append(f1_c)
        losses_cap.append(loss_c)

        print(f"[Hetero-Finance] Round {r+1}/{rounds}: Simple Acc={acc_s:.4f}, F1={f1_s:.4f} | Cap-Weighted Acc={acc_c:.4f}, F1={f1_c:.4f}")

    return (accs_simple, f1s_simple, losses_simple, accs_cap, f1s_cap, losses_cap)

# Run the experiment
finance_wrappers = [XGBWrap, TabNetWrap, RFWrap]
hetfin_results = run_hetero_finance_distill(
    X_train, y_train, X_distill, y_distill, X_test, y_test,
    X_train_np, y_train_np, X_distill_np, y_distill_np, X_test_np, y_test_np,
    finance_wrappers, num_clients=3, rounds=10, local_epochs=5, lr=0.01
)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [05:57:09] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


[Hetero-Finance] Round 1/10: Simple Acc=0.6329, F1=0.0006 | Cap-Weighted Acc=0.8085, F1=0.0011


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [05:59:31] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


[Hetero-Finance] Round 2/10: Simple Acc=0.6329, F1=0.0006 | Cap-Weighted Acc=0.8085, F1=0.0011


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [06:01:49] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [06:08:46] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


[Hetero-Finance] Round 6/10: Simple Acc=0.6329, F1=0.0006 | Cap-Weighted Acc=0.8085, F1=0.0011


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [06:11:04] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


[Hetero-Finance] Round 7/10: Simple Acc=0.6329, F1=0.0006 | Cap-Weighted Acc=0.8085, F1=0.0011


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [06:13:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [1]:
# Unpack results
hom_rounds = len(hom_accs)
hetmlp_accs_s, hetmlp_f1s_s, hetmlp_losses_s, hetmlp_accs_c, hetmlp_f1s_c, hetmlp_losses_c = hetmlp_results
hetfin_accs_s, hetfin_f1s_s, hetfin_losses_s, hetfin_accs_c, hetfin_f1s_c, hetfin_losses_c = hetfin_results

rounds = range(1, hom_rounds+1)

plt.figure(figsize=(16,5))
plt.subplot(1,3,1)
plt.plot(rounds, hom_accs, label='Homog FedAvg')
plt.plot(rounds, hetmlp_accs_s, label='Hetero-MLP (simple distill)')
plt.plot(rounds, hetmlp_accs_c, label='Hetero-MLP (cap-weight distill)')
plt.plot(range(1,len(hetfin_accs_s)+1), hetfin_accs_s, label='Hetero-Fin (simple)')
plt.plot(range(1,len(hetfin_accs_c)+1), hetfin_accs_c, label='Hetero-Fin (cap-weight)')
plt.xlabel("Round"); plt.ylabel("Test Accuracy"); plt.title("Accuracy vs Rounds"); plt.legend()

plt.subplot(1,3,2)
plt.plot(rounds, hom_f1s, label='Homog FedAvg')
plt.plot(rounds, hetmlp_f1s_s, label='Hetero-MLP (simple)')
plt.plot(rounds, hetmlp_f1s_c, label='Hetero-MLP (cap)')
plt.plot(range(1,len(hetfin_f1s_s)+1), hetfin_f1s_s, label='Hetero-Fin (simple)')
plt.plot(range(1,len(hetfin_f1s_c)+1), hetfin_f1s_c, label='Hetero-Fin (cap)')
plt.xlabel("Round"); plt.ylabel("Test F1"); plt.title("F1 vs Rounds"); plt.legend()

plt.subplot(1,3,3)
plt.plot(rounds, hom_losses, label='Homog FedAvg')
plt.plot(rounds, hetmlp_losses_s, label='Hetero-MLP (simple)')
plt.plot(rounds, hetmlp_losses_c, label='Hetero-MLP (cap)')
plt.plot(range(1,len(hetfin_losses_s)+1), hetfin_losses_s, label='Hetero-Fin (simple)')
plt.plot(range(1,len(hetfin_losses_c)+1), hetfin_losses_c, label='Hetero-Fin (cap)')
plt.xlabel("Round"); plt.ylabel("BCE Loss (test)"); plt.title("Test Loss vs Rounds"); plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'hom_accs' is not defined